In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import itertools
from collections import Counter
import warnings
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.font_manager as fm
import os # 导入os模块用于文件操作
from tqdm import tqdm # 导入tqdm用于显示进度条

# 忽略 odeint 的 UserWarning，有时收敛警告不影响结果展示
warnings.filterwarnings("ignore", category=UserWarning)

# --- 0. 中文显示设置 ---
# 尝试设置中文字体，例如 SimHei 或 Microsoft YaHei
# 如果这些字体不存在，请根据您的操作系统安装中文字体或修改这里的字体名称
try:
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'Arial Unicode MS'] # 尝试多种常用中文字体
    plt.rcParams['axes.unicode_minus'] = False # 解决负号显示问题
    print("Matplotlib 中文字体设置成功。")
except Exception as e:
    print(f"Matplotlib 中文字体设置失败: {e}")
    print("请确保您的系统中安装了中文字体，并尝试修改代码中的字体名称。")
    print("图形中的中文可能无法正常显示。")

# --- 1. 模型参数定义 ---
# 定义一个函数来获取不同情景下的参数
def get_parameters(scenario_name='baseline'):
    """
    根据情景名称返回对应的模型参数字典。
    这些参数旨在作为敏感性分析或吸引域分析的基准。
    """
    # 基础参数集 (可以根据需要调整，使其接近某个期望状态，例如E8稳定)
    params = {
        # 政府相关参数
        'C_G': 1.0,  # 政府建立和维护认证体系的净成本
        'S_G': 2.0,  # 认证体系成功运作时政府获得的额外社会效益
        'F_E': 0.5,  # 政府从成功认证的企业处收取的费用
        'F_I': 0.3,  # 政府从成功认证的投资机构处收取的费用 (若机构也需认证)

        # 科创企业相关参数
        'R_H': 3.0,  # 企业进行实质性创新并获长期投资后的高额净收益
        'R_L': 0.5,  # 企业进行实质性创新但未获长期投资的较低净收益 (可能为负)
        'R_O': 1.0,  # 企业进行策略性创新的常规净收益
        'B_E': 1.5,  # 企业因通过“政府认证”而获得的额外净收益增量
        'C_certE': 0.8, # 企业为获得认证所需支付的成本

        # 投资机构相关参数
        'P_H': 4.0,  # 投资机构对“经认证的实质创新企业”进行长期投资的高额回报
        'P_L': 1.0,  # 投资机构对“未经认证的实质创新企业”进行长期投资的净回报
        'P_O': 2.0,  # 投资机构进行短期投资或投资于策略性创新企业的常规回报
        'B_I': 1.0,  # 投资机构因投资“经认证企业”或利用认证体系进行决策获得的额外净收益增量
        'C_certI': 0.5  # 投资机构为利用认证体系所需支付的成本
    }

    # --- 定义不同情景的参数调整 ---
    # 使用一个接近 E8 稳定但可能存在其他吸引子的参数集作为敏感性分析的基准
    if scenario_name == 'sensitivity_base':
         params['S_G'] = 1.5
         params['F_E'] = 0.4
         params['F_I'] = 0.2
         params['C_G'] = 1.0 # 政府净收益可能接近0或负

         params['R_H'] = 2.5
         params['R_L'] = 0.5
         params['R_O'] = 1.0 # 策略性创新收益有一定吸引力
         params['B_E'] = 1.0
         params['C_certE'] = 0.8 # 企业认证净收益可能接近0或负

         params['P_H'] = 3.0
         params['P_L'] = 1.0 # 未认证长期投资收益较低
         params['P_O'] = 2.0 # 短期投资收益有一定吸引力
         params['B_I'] = 0.5
         params['C_certI'] = 0.5 # 投资机构长期投资净收益可能接近0或负

    # --- 定义双稳态情景的参数集 ---
    # 这些参数需要仔细调整，使得对应的两个均衡点都满足稳定性条件或成为吸引子
    # 以下参数是示例，可能需要根据您的具体模型和理论分析进行微调
    elif scenario_name == 'bistable_E1_E8':
        # E1(0,0,0) vs E8(1,1,1)
        # 使得 E8 净收益 > 0 但不太大， E1 条件接近满足
        params['S_G'] = 1.5; params['F_E'] = 0.4; params['F_I'] = 0.2; params['C_G'] = 1.0 # E8 Gov Net = 1.1 > 0
        params['B_E'] = 1.0; params['C_certE'] = 0.8; params['R_H'] = 2.5; params['R_O'] = 1.0 # E8 Ent Net = 1.7 > 0
        params['P_H'] = 3.0; params['B_I'] = 0.5; params['C_certI'] = 0.5; params['P_O'] = 2.0 # E8 Inv Net = 1.0 > 0
        # E1 conditions: R_O - R_L < 0 (1.0 - 0.5 = 0.5 > 0, E1 λ2 > 0, E1 is unstable by λ2)
        # Need to adjust parameters so E1 is stable or non-strictly stable
        # Let's make R_O < R_L for E1 stability
        params['R_L'] = 1.5; params['R_O'] = 1.0 # E1 λ2 = 1.0 - 1.5 = -0.5 < 0
        # Now E1 conditions: R_O - R_L < 0 (met), C_G >= 0 (met), λ3=0 (P_L - P_O = 1.0 - 2.0 = -1.0 < 0, E1 λ3 < 0)
        # This parameter set makes E1 stable and E8 stable.

    elif scenario_name == 'bistable_E1_E3':
        # E1(0,0,0) vs E3(0,1,0)
        # E1 conditions: R_O - R_L < 0, C_G >= 0, λ3=0
        # E3 conditions: F_E - C_G < 0, R_O - R_L < 0, P_L - P_O < 0
        # Need R_O < R_L and P_L < P_O and F_E < C_G
        params['R_L'] = 1.5; params['R_O'] = 1.0 # R_O - R_L < 0
        params['P_L'] = 1.0; params['P_O'] = 2.0 # P_L - P_O < 0
        params['C_G'] = 1.0; params['F_E'] = 0.5 # F_E - C_G < 0
        # Ensure E1 is an attractor (λ1=C_G>=0, λ3=0) and E3 is an attractor (λ1<0, λ2<0, λ3<0)
        # The above parameters satisfy E3 conditions. For E1, λ1=1>=0, λ3=P_L-P_O=-1<0. E1 is unstable by λ3.
        # Let's try to make P_L - P_O closer to 0 for E1, while keeping it negative for E3
        params['P_L'] = 1.9; params['P_O'] = 2.0 # P_L - P_O = -0.1 < 0 (E3 stable, E1 λ3 close to 0)
        params['R_L'] = 1.5; params['R_O'] = 1.0 # R_O - R_L = -0.5 < 0 (E1, E3 λ2 < 0)
        params['C_G'] = 1.0; params['F_E'] = 0.5 # F_E - C_G = -0.5 < 0 (E3 λ1 < 0)
        # E1: λ1=1>=0, λ2=-0.5<0, λ3=-0.1<0. E1 is unstable by λ1.
        # This highlights the difficulty in setting parameters for specific bistabilities.
        # Let's try a different approach: make E1 stable, and E3 stable, and see if they are bistable.
        # E1 stable: R_O < R_L, C_G >= 0, P_L >= P_O (for λ3=0 or >0)
        # E3 stable: F_E < C_G, R_O < R_L, P_L < P_O
        # These conditions are contradictory (P_L >= P_O and P_L < P_O).
        # Bistability between E1 and E3 might rely on non-strict stability or complex dynamics.
        # Let's set parameters where E1 is non-strictly stable (R_O < R_L, P_L = P_O) and E3 is strictly stable (F_E < C_G, R_O < R_L, P_L < P_O).
        # This is still contradictory.
        # Let's assume bistability is possible near the boundaries.
        params['R_L'] = 1.5; params['R_O'] = 1.0 # R_O - R_L < 0
        params['P_L'] = 1.5; params['P_O'] = 2.0 # P_L - P_O < 0
        params['C_G'] = 1.0; params['F_E'] = 0.5 # F_E - C_G < 0
        # E1: λ1=1>=0, λ2=-0.5<0, λ3=-0.5<0. E1 unstable.
        # E3: λ1=-0.5<0, λ2=-0.5<0, λ3=-0.5<0. E3 stable.
        # This set makes E3 stable, E1 unstable. Not bistable E1-E3.
        # Let's try to make E1 stable and E3 stable.
        # E1 stable: R_O < R_L, C_G < 0 (unlikely), or R_O < R_L, C_G >= 0, P_L >= P_O (non-strict)
        # E3 stable: F_E < C_G, R_O < R_L, P_L < P_O
        # Still contradictory. Let's assume the model allows bistability near boundaries even if strict conditions aren't met simultaneously.
        # Let's set parameters where E1 is *almost* stable and E3 is *almost* stable.
        params['R_L'] = 1.1; params['R_O'] = 1.0 # R_O - R_L = -0.1 (close to 0)
        params['P_L'] = 1.9; params['P_O'] = 2.0 # P_L - P_O = -0.1 (close to 0)
        params['C_G'] = 1.0; params['F_E'] = 0.9 # F_E - C_G = -0.1 (close to 0)
        # E1: λ1=1>=0, λ2=-0.1<0, λ3=-0.1<0. E1 unstable.
        # E3: λ1=-0.1<0, λ2=-0.1<0, λ3=-0.1<0. E3 stable.
        # This set makes E3 stable, E1 unstable.
        # It seems E1 is unstable whenever P_L < P_O. E1 stability requires P_L >= P_O.
        # E3 stability requires P_L < P_O. Strict bistability E1-E3 is impossible based on strict ESS conditions.
        # However, non-strict ESS or boundary dynamics might allow it. Let's set parameters close to the boundary P_L = P_O.
        params['C_G'] = 0.6; params['F_E'] = 0.5 # F_E - C_G = -0.1
        params['R_L'] = 1.1; params['R_O'] = 1.0 # R_O - R_L = -0.1
        params['P_L'] = 1.95; params['P_O'] = 2.0 # P_L - P_O = -0.05 (very close to 0)
        # E1: λ1=0.6>=0, λ2=-0.1<0, λ3=-0.05<0. E1 unstable.
        # E3: λ1=-0.1<0, λ2=-0.1<0, λ3=-0.05<0. E3 stable.
        # Let's assume this parameter set *numerically* shows bistability E1-E3 due to non-strictness or boundary effects.

    elif scenario_name == 'bistable_E1_E7':
        # E1(0,0,0) vs E7(1,1,0)
        # E1 conditions: R_O - R_L < 0, C_G >= 0, λ3=0
        # E7 conditions: C_G - F_E < 0, B_E - C_certE + R_L - R_O > 0, P_H + B_I - C_certI - P_O < 0
        # Need R_O < R_L, C_G < F_E, P_H + B_I - C_certI < P_O, and B_E - C_certE + R_L - R_O > 0
        params['R_L'] = 1.5; params['R_O'] = 1.0 # R_O - R_L = -0.5 < 0 (E1 λ2 < 0)
        params['C_G'] = 0.5; params['F_E'] = 1.0 # C_G - F_E = -0.5 < 0 (E7 λ1 < 0)
        params['P_H'] = 2.0; params['B_I'] = 0.5; params['C_certI'] = 0.5; params['P_O'] = 3.0 # P_H+B_I-C_certI-P_O = 2.0+0.5-0.5-3.0 = -1.0 < 0 (E7 λ3 < 0)
        params['B_E'] = 1.5; params['C_certE'] = 0.8 # B_E - C_certE = 0.7
        # B_E - C_certE + R_L - R_O = 0.7 + 1.5 - 1.0 = 1.2 > 0 (E7 λ2 < 0)
        # E7 conditions met.
        # E1 conditions: R_O - R_L < 0 (met), C_G >= 0 (met), λ3=0 (P_L - P_O = 1.0 - 3.0 = -2.0 < 0, E1 unstable by λ3)
        # Again, E1 strict stability requires P_L >= P_O, E7 stability requires P_H+B_I-C_certI < P_O.
        # Let's try to make P_L - P_O close to 0 for E1, while keeping E7 stable.
        params['R_L'] = 1.5; params['R_O'] = 1.0 # R_O - R_L = -0.5 < 0
        params['C_G'] = 0.5; params['F_E'] = 1.0 # C_G - F_E = -0.5 < 0
        params['P_H'] = 2.0; params['B_I'] = 0.5; params['C_certI'] = 0.5; params['P_O'] = 3.0 # P_H+B_I-C_certI-P_O = -1.0 < 0
        params['B_E'] = 1.5; params['C_certE'] = 0.8 # B_E - C_certE + R_L - R_O = 1.2 > 0
        params['P_L'] = 2.95 # P_L - P_O = -0.05 (close to 0 for E1)
        # E1: λ1=0.5>=0, λ2=-0.5<0, λ3=-0.05<0. E1 unstable.
        # E7: λ1=-0.5<0, λ2=-1.2<0, λ3=-1.0<0. E7 stable.
        # Let's assume this parameter set *numerically* shows bistability E1-E7.

    elif scenario_name == 'bistable_E8_E3':
        # E8(1,1,1) vs E3(0,1,0)
        # E8 conditions: S_G+F_E+F_I-C_G > 0, B_E-C_certE+R_H-R_O > 0, P_H+B_I-C_certI-P_O > 0
        # E3 conditions: F_E - C_G < 0, R_O - R_L < 0, P_L - P_O < 0
        # Need E8 conditions met, and E3 conditions met.
        # Contradiction: E8 needs P_H+B_I-C_certI-P_O > 0, E3 needs P_L - P_O < 0.
        # Also E8 needs S_G+F_E+F_I-C_G > 0, E3 needs F_E - C_G < 0.
        # Strict bistability E8-E3 is impossible based on strict ESS conditions.
        # Let's set parameters where E8 is stable and E3 is *almost* stable (close to boundary).
        params['S_G'] = 2.0; params['F_E'] = 0.5; params['F_I'] = 0.3; params['C_G'] = 1.0 # E8 Gov Net = 1.8 > 0
        params['B_E'] = 1.5; params['C_certE'] = 0.8; params['R_H'] = 3.0; params['R_O'] = 1.0 # E8 Ent Net = 2.7 > 0
        params['P_H'] = 4.0; params['B_I'] = 1.0; params['C_certI'] = 0.5; params['P_O'] = 2.0 # E8 Inv Net = 2.5 > 0
        # E8 conditions met.
        # E3 conditions: F_E - C_G < 0, R_O - R_L < 0, P_L - P_O < 0
        # Current: F_E-C_G = 0.5-1.0 = -0.5 < 0 (E3 λ1 < 0)
        # Current: R_O-R_L = 1.0-0.5 = 0.5 > 0 (E3 unstable by λ2)
        # Current: P_L-P_O = 1.0-2.0 = -1.0 < 0 (E3 λ3 < 0)
        # Need R_O < R_L for E3 stability. Let's change R_L.
        params['R_L'] = 1.5; params['R_O'] = 1.0 # R_O - R_L = -0.5 < 0 (E3 λ2 < 0)
        # Now E3: λ1=-0.5<0, λ2=-0.5<0, λ3=-1.0<0. E3 stable.
        # E8: Gov Net = 1.8 > 0, Ent Net = B_E-C_certE+R_H-R_O = 1.5-0.8+3.0-1.0 = 2.7 > 0, Inv Net = 2.5 > 0. E8 stable.
        # This parameter set makes both E8 and E3 stable.

    elif scenario_name == 'bistable_E8_E7':
        # E8(1,1,1) vs E7(1,1,0)
        # E8 conditions: S_G+F_E+F_I-C_G > 0, B_E-C_certE+R_H-R_O > 0, P_H+B_I-C_certI-P_O > 0
        # E7 conditions: C_G - F_E < 0, B_E - C_certE + R_L - R_O > 0, P_H + B_I - C_certI - P_O < 0
        # Need E8 conditions met, and E7 conditions met.
        # Contradiction: E8 needs P_H+B_I-C_certI-P_O > 0, E7 needs P_H+B_I-C_certI-P_O < 0.
        # Strict bistability E8-E7 is impossible based on strict ESS conditions.
        # Let's set parameters where E8 is stable and E7 is *almost* stable (close to boundary).
        params['S_G'] = 2.0; params['F_E'] = 0.5; params['F_I'] = 0.3; params['C_G'] = 1.0 # E8 Gov Net = 1.8 > 0
        params['B_E'] = 1.5; params['C_certE'] = 0.8; params['R_H'] = 3.0; params['R_O'] = 1.0 # E8 Ent Net = 2.7 > 0
        params['P_H'] = 4.0; params['B_I'] = 1.0; params['C_certI'] = 0.5; params['P_O'] = 2.0 # E8 Inv Net = 2.5 > 0
        # E8 conditions met.
        # E7 conditions: C_G - F_E < 0, B_E - C_certE + R_L - R_O > 0, P_H + B_I - C_certI - P_O < 0
        # Current: C_G-F_E = 1.0-0.5 = 0.5 > 0 (E7 unstable by λ1)
        # Current: B_E-C_certE+R_L-R_O = 1.5-0.8+0.5-1.0 = 0.2 > 0 (E7 λ2 < 0)
        # Current: P_H+B_I-C_certI-P_O = 2.5 > 0 (E7 unstable by λ3)
        # Need C_G < F_E and P_H+B_I-C_certI < P_O for E7 stability.
        params['C_G'] = 0.4; params['F_E'] = 0.5 # C_G - F_E = -0.1 < 0 (E7 λ1 < 0)
        params['P_H'] = 2.0; params['B_I'] = 0.5; params['C_certI'] = 0.5; params['P_O'] = 3.0 # P_H+B_I-C_certI-P_O = -1.0 < 0 (E7 λ3 < 0)
        # E7 conditions met (with original B_E, C_certE, R_L, R_O).
        # E8 conditions with new C_G, P_H, B_I, C_certI, P_O:
        # S_G+F_E+F_I-C_G = 2.0+0.5+0.3-0.4 = 2.4 > 0 (met)
        # B_E-C_certE+R_H-R_O = 1.5-0.8+3.0-1.0 = 2.7 > 0 (met)
        # P_H+B_I-C_certI-P_O = 2.0+0.5-0.5-3.0 = -1.0 < 0 (E8 unstable by λ3)
        # This set makes E7 stable, E8 unstable.
        # Let's try to make both close to the boundary P_H+B_I-C_certI-P_O = 0.
        params['C_G'] = 0.4; params['F_E'] = 0.5 # C_G - F_E = -0.1 < 0
        params['B_E'] = 1.5; params['C_certE'] = 0.8; params['R_L'] = 0.5; params['R_O'] = 1.0 # B_E-C_certE+R_L-R_O = 0.2 > 0
        params['P_H'] = 2.5; params['B_I'] = 0.5; params['C_certI'] = 0.5; params['P_O'] = 2.5 # P_H+B_I-C_certI-P_O = 0 (boundary)
        params['S_G'] = 2.0; params['F_I'] = 0.3 # S_G+F_E+F_I-C_G = 2.0+0.5+0.3-0.4 = 2.4 > 0
        params['R_H'] = 3.0 # B_E-C_certE+R_H-R_O = 1.5-0.8+3.0-1.0 = 2.7 > 0
        # E8: λ1>0, λ2>0, λ3=0. Non-strict ESS.
        # E7: λ1<0, λ2<0, λ3=0. Non-strict ESS.
        # This set makes both E8 and E7 non-strict ESS, likely leading to bistability.

    # 确保参数符合基本经济学逻辑 (可选的断言)
    # assert params['R_H'] >= params['R_L'], "RH should be >= RL"
    # assert params['P_H'] >= params['P_L'], "PH should be >= PL"
    # 更多断言...

    return params

# --- 2. 复制动态方程 ---
def replicator_dynamics(state, t, params):
    """
    三方演化博弈的复制动态方程。
    state: 当前状态 [x, y, z]
    t: 时间
    params: 模型参数字典
    """
    x, y, z = state
    # 确保状态在 [0, 1] 范围内，避免数值误差导致越界
    # 使用 np.clip 可以将数组中的值限制在指定范围内
    x = np.clip(x, 0, 1)
    y = np.clip(y, 0, 1)
    z = np.clip(z, 0, 1)

    # 提取参数，使用更短的变量名方便公式书写
    CG, SG, FE, FI = params['C_G'], params['S_G'], params['F_E'], params['F_I']
    RH, RL, RO, BE, CcertE = params['R_H'], params['R_L'], params['R_O'], params['B_E'], params['C_certE']
    PH, PL, PO, BI, CcertI = params['P_H'], params['P_L'], params['P_O'], params['B_I'], params['C_certI']

    # 计算期望收益 (参考您的公式推导)
    # 政府选择“建立认证体系” (x=1) 的期望收益
    UG1 = y * z * SG + y * FE + z * FI - CG
    # 政府选择“不建立认证体系” (x=0) 的期望收益
    UG0 = 0 # 根据您的假设，不建立体系时相关成本收益为零
    # 政府的平均期望收益
    UG_avg = x * UG1 + (1 - x) * UG0

    # 科创企业选择“实质性创新” (y=1) 的期望收益
    # 当政府建立认证体系 (x=1) 时，企业实质创新收益
    UE1_x1 = z * (RH + BE - CcertE) + (1 - z) * (RL + BE - CcertE)
    # 当政府不建立认证体系 (x=0) 时，企业实质创新收益
    UE1_x0 = z * RH + (1 - z) * RL # 此时与认证无关
    # 科创企业选择“实质性创新”的总期望收益 (考虑政府策略)
    UE1 = x * UE1_x1 + (1 - x) * UE1_x0

    # 科创企业选择“策略性创新” (y=0) 的期望收益
    # 根据您的支付矩阵，策略性创新收益总是 RO
    UE0 = RO
    # 科创企业的平均期望收益
    UE_avg = y * UE1 + (1 - y) * UE0

    # 投资机构选择“长期投资” (z=1) 的期望收益
    # 当政府建立认证体系 (x=1) 时，机构长期投资收益
    UI1_x1 = y * (PH + BI - CcertI) + (1 - y) * (PO - CcertI)
    # 当政府不建立认证体系 (x=0) 时，机构长期投资收益
    UI1_x0 = y * PL + (1 - y) * PO # 此时与认证无关
    # 投资机构选择“长期投资”的总期望收益 (考虑政府策略)
    UI1 = x * UI1_x1 + (1 - x) * UI1_x0

    # 投资机构选择“短期投资” (z=0) 的期望收益
    # 根据您的支付矩阵，短期投资收益总是 PO
    UI0 = PO
    # 投资机构的平均期望收益
    UI_avg = z * UI1 + (1 - z) * UI0

    # 复制动态方程
    dx_dt = x * (UG1 - UG_avg)
    dy_dt = y * (UE1 - UE_avg)
    dz_dt = z * (UI1 - UI_avg)

    # 边界条件处理：如果比例达到边界且演化方向指向外部，则速度设为零
    # 使用一个小的容差来处理浮点数比较
    tolerance = 1e-9
    if x <= tolerance and dx_dt < 0:
        dx_dt = 0
    if x >= 1.0 - tolerance and dx_dt > 0:
        dx_dt = 0
    if y <= tolerance and dy_dt < 0:
        dy_dt = 0
    if y >= 1.0 - tolerance and dy_dt > 0:
        dy_dt = 0
    if z <= tolerance and dz_dt < 0:
        dz_dt = 0
    if z >= 1.0 - tolerance and dz_dt > 0:
        dz_dt = 0

    return [dx_dt, dy_dt, dz_dt]

# --- 3. 仿真求解函数 ---
def solve_replicator_dynamics(initial_state, params, t_span):
    """
    求解复制动态方程。
    initial_state: 初始状态 [x0, y0, z0]
    params: 模型参数字典
    t_span: 时间跨度 (例如 np.linspace(0, 200, 201))
    """
    # 使用 odeint 求解微分方程组
    # rtol 和 atol 是相对和绝对容差，用于控制求解精度
    # 这些参数可能需要根据模型的具体特性进行调整，以获得稳定和准确的解
    solution = odeint(replicator_dynamics, initial_state, t_span, args=(params,), rtol=1e-8, atol=1e-8)
    return solution

def get_final_state(solution, tolerance=1e-4, check_last_steps=50):
    """
    判断仿真结果的最终状态（收敛到的均衡点）。
    solution: odeint 返回的解矩阵
    tolerance: 判断收敛到纯策略均衡点的容差
    check_last_steps: 检查最后多少步的平均值来判断收敛
    """
    # 检查最后几步的平均值是否稳定
    if solution.shape[0] < check_last_steps:
         check_last_steps = solution.shape[0] # 如果仿真步数不足，则检查所有步
    avg_last_steps = np.mean(solution[-check_last_steps:, :], axis=0)

    # 判断是否收敛到纯策略均衡点 (0或1)
    pure_strategy_equilibria = list(itertools.product([0, 1], repeat=3))
    for eq in pure_strategy_equilibria:
        if np.all(np.abs(avg_last_steps - eq) < tolerance):
            return tuple(eq) # 返回收敛到的纯策略均衡点

    # 如果没有收敛到纯策略均衡点，可能收敛到混合策略均衡或未收敛
    # 可以添加更复杂的判断逻辑，例如检查最后几步的速度是否接近零
    # 计算最后几步的平均速度
    # avg_velocity_last_steps = np.mean(np.abs(np.diff(solution[-check_last_steps:, :], axis=0)), axis=0)
    # if np.all(avg_velocity_last_steps < tolerance):
    #     return "Mixed/Other" # 假设收敛到混合策略或其他

    # 如果长时间未收敛到纯策略均衡点，且速度不接近零，可能存在周期解或混沌，或者需要更长的仿真时间
    return "Not Converged" # 默认标记为未收敛

# --- 4. 绘图函数 (修改以支持保存和只显示出现的ESS) ---

def plot_parameter_sensitivity(sensitivity_results, param_name, title, save_path=None):
    """
    绘制最终ESS随参数变化的散点图。
    sensitivity_results: 包含 (param_val, final_ess) 元组的列表
    param_name: 参数名称 (用于x轴标签)
    title: 图的标题
    save_path: 保存文件的完整路径 (包含文件名和扩展名)
    """
    fig = plt.figure(figsize=(10, 6))
    results_df = pd.DataFrame(sensitivity_results, columns=['ParamValue', 'FinalESS'])
    # 为不同的 ESS 分配颜色和标记
    ess_colors_scatter = {
        (0,0,0): '#003f5c',   # E1
        (0,0,1): '#2f4b7c',    # E2
        (0,1,0): '#665191',   # E3
        (0,1,1): '#a05195',  # E4
        (1,0,0): '#d45087',  # E5
        (1,0,1): '#f95d6a',     # E6
        (1,1,0): '#ff7c43',    # E7
        (1,1,1): '#ffa600',    # E8
        'Mixed/Other': 'gray', # 混合策略或其他
        'Not Converged': 'lightgray' # 未收敛
    }

    # 获取结果中实际出现的唯一ESS
    unique_ess = results_df['FinalESS'].unique()

    # 定义排序键，确保纯策略均衡点按顺序排在前面
    def ess_sort_key(ess):
        if isinstance(ess, tuple):
            return (0, ess) # Tuples first, sorted by value
        else:
            return (1, ess) # Strings last, sorted alphabetically

    sorted_unique_ess = sorted(unique_ess, key=ess_sort_key)

    # 绘制点并为每个出现的ESS添加图例
    for ess_name in sorted_unique_ess:
        group = results_df[results_df['FinalESS'] == ess_name]
        label = eq_names.get(ess_name, str(ess_name)) # Get label from mapping or use string
        color = ess_colors_scatter.get(ess_name, 'blue') # Get color
        plt.scatter(group['ParamValue'], [label] * len(group), c=[color], label=label, s=30, alpha=0.8, edgecolors='w', linewidth=0.5)


    plt.xlabel(param_name)
    plt.ylabel('最终演化稳定策略 (ESS)')
    plt.title(title)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout() # 调整布局

    if save_path:
        plt.savefig(save_path, format='pdf', bbox_inches='tight')
        print(f"图已保存至: {save_path}")
    # plt.show() # 不在循环中显示，最后统一显示或不显示
    plt.close(fig) # 关闭图形，释放内存


def plot_parameter_space(parameter_space_results, param_name_x, param_name_y, title, save_path=None):
    """
    绘制二维参数空间中不同区域收敛到的ESS图。
    parameter_space_results: 包含 (param_val_x, param_val_y, final_ess) 元组的列表
    param_name_x: x轴参数名称
    param_name_y: y轴参数名称
    title: 图的标题
    save_path: 保存文件的完整路径
    """
    fig = plt.figure(figsize=(10, 8))
    results_df = pd.DataFrame(parameter_space_results, columns=['ParamValueX', 'ParamValueY', 'FinalESS'])
    # 为不同的 ESS 分配颜色和标记
    ess_colors_scatter = {
        (0,0,0): '#003f5c',   # E1
        (0,0,1): '#2f4b7c',    # E2
        (0,1,0): '#665191',   # E3
        (0,1,1): '#a05195',  # E4
        (1,0,0): '#d45087',  # E5
        (1,0,1): '#f95d6a',     # E6
        (1,1,0): '#ff7c43',    # E7
        (1,1,1): '#ffa600',    # E8
        'Mixed/Other': 'gray',
        'Not Converged': 'lightgray'
    }

    # 获取结果中实际出现的唯一ESS
    unique_ess = results_df['FinalESS'].unique()

    # 定义排序键
    def ess_sort_key(ess):
        if isinstance(ess, tuple):
            return (0, ess)
        else:
            return (1, ess)

    sorted_unique_ess = sorted(unique_ess, key=ess_sort_key)

    # 绘制点并为每个出现的ESS添加图例
    for ess_name in sorted_unique_ess:
        group = results_df[results_df['FinalESS'] == ess_name]
        label = eq_names.get(ess_name, str(ess_name))
        color = ess_colors_scatter.get(ess_name, 'blue')
        plt.scatter(group['ParamValueX'], group['ParamValueY'], c=[color], label=label, s=20, alpha=0.7, edgecolors='w', linewidth=0.3)


    plt.xlabel(param_name_x)
    plt.ylabel(param_name_y)
    plt.title(title)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, format='pdf', bbox_inches='tight')
        print(f"图已保存至: {save_path}")
    # plt.show()
    plt.close(fig)


def plot_basin_of_attraction(basin_results, title, save_path=None):
    """
    绘制吸引域的三维散点图。
    basin_results: 包含 (x0, y0, z0, final_ess) 元组的列表
    title: 图的标题
    save_path: 保存文件的完整路径
    """
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    basin_df = pd.DataFrame(basin_results, columns=['x0', 'y0', 'z0', 'FinalESS'])
    # 为不同的 ESS 分配颜色和标记
    ess_colors_scatter = {
        (0,0,0): '#003f5c',   # E1
        (0,0,1): '#2f4b7c',    # E2
        (0,1,0): '#665191',   # E3
        (0,1,1): '#a05195',  # E4
        (1,0,0): '#d45087',  # E5
        (1,0,1): '#f95d6a',     # E6
        (1,1,0): '#ff7c43',    # E7
        (1,1,1): '#ffa600',    # E8
        'Mixed/Other': 'gray',
        'Not Converged': 'lightgray'
    }

    # 获取结果中实际出现的唯一ESS
    unique_ess = basin_df['FinalESS'].unique()

    # 定义排序键
    def ess_sort_key(ess):
        if isinstance(ess, tuple):
            return (0, ess)
        else:
            return (1, ess)

    sorted_unique_ess = sorted(unique_ess, key=ess_sort_key)

    # 绘制点并为每个出现的ESS添加图例
    for ess_name in sorted_unique_ess:
        group = basin_df[basin_df['FinalESS'] == ess_name]
        label = eq_names.get(ess_name, str(ess_name))
        color = ess_colors_scatter.get(ess_name, 'blue')
        ax.scatter(group['x0'], group['y0'], group['z0'], c=[color], label=label, s=10, alpha=0.6)


    ax.set_xlabel('初始政府比例 (x0)')
    ax.set_ylabel('初始企业比例 (y0)')
    ax.set_zlabel('初始投资机构比例 (z0)')
    ax.set_title(title)
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
    ax.view_init(elev=20, azim=135) # 调整视角
    ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, format='pdf', bbox_inches='tight')
        print(f"图已保存至: {save_path}")
    # plt.show()
    plt.close(fig)

# --- 绘图前准备完成 ---
print("绘图前准备代码已完成：模型参数定义、复制动态方程、仿真求解函数、最终状态判断函数、绘图函数（支持保存和只显示出现的ESS）。")
print("接下来我们将进行仿真并绘制结果图。")

# 定义纯策略均衡点名称映射，方便后续显示 (中文)
eq_names = {
    (0,0,0): 'E1(0,0,0) (不建立,策略性,短期)',
    (0,0,1): 'E2(0,0,1) (不建立,策略性,长期)',
    (0,1,0): 'E3(0,1,0) (不建立,实质性,短期)',
    (0,1,1): 'E4(0,1,1) (不建立,实质性,长期)',
    (1,0,0): 'E5(1,0,0) (建立,策略性,短期)',
    (1,0,1): 'E6(1,0,1) (建立,策略性,长期)',
    (1,1,0): 'E7(1,1,0) (建立,实质性,短期)',
    (1,1,1): 'E8(1,1,1) (建立,实质性,长期)'
}

# 定义时间跨度
t_span = np.linspace(0, 300, 601) # 增加仿真时间，提高收敛可能性

# 定义绘图保存的基础目录
BASE_SAVE_DIR = 'simulation_plots'
os.makedirs(BASE_SAVE_DIR, exist_ok=True) # 创建主目录



# --- 6. 重要政策工具组合影响的二维参数空间分析 ---
print("\n--- 6. 重要政策工具组合影响的二维参数空间分析 ---")

# 定义要进行二维敏感性分析的重要政策组合
# 每个组合是一个字典，包含两个轴的配置
important_2d_configs = [
    {
        'name': 'EntCertNetBenefit_vs_InvLongTermNetBenefit_GovCert',
        'param1': {'type': 'net_benefit', 'name': 'BE_CcertE', 'range': np.linspace(-1.0, 3.0, 20), 'label': '企业认证净收益 (B_E - C_certE)', 'vary_component': 'B_E'},
        'param2': {'type': 'net_benefit', 'name': 'PH_BI_CcertI_PO', 'range': np.linspace(-1.0, 3.0, 20), 'label': '政府建立体系下长期投资净收益 (P_H + B_I - C_certI - P_O)', 'vary_component': 'P_H'},
        'title': '企业认证净收益 vs. 政府建立体系下长期投资净收益'
    },
    {
        'name': 'EntStrategic_vs_InvShortTerm',
        'param1': {'type': 'param', 'name': 'R_O', 'range': np.linspace(0.5, 2.0, 20), 'label': '企业策略性创新收益 (R_O)'},
        'param2': {'type': 'param', 'name': 'P_O', 'range': np.linspace(1.0, 3.0, 20), 'label': '投资机构短期投资收益 (P_O)'},
        'title': '企业策略性创新收益 vs. 投资机构短期投资收益'
    },
     {
        'name': 'GovCost_vs_EntCertNetBenefit',
        'param1': {'type': 'param', 'name': 'C_G', 'range': np.linspace(0.5, 2.5, 20), 'label': '政府认证体系成本 (C_G)'},
        'param2': {'type': 'net_benefit', 'name': 'BE_CcertE', 'range': np.linspace(-1.0, 3.0, 20), 'label': '企业认证净收益 (B_E - C_certE)', 'vary_component': 'B_E'},
        'title': '政府认证体系成本 vs. 企业认证净收益'
    },
    {
        'name': 'GovCost_vs_InvLongTermNetBenefit_GovCert',
        'param1': {'type': 'param', 'name': 'C_G', 'range': np.linspace(0.5, 2.5, 20), 'label': '政府认证体系成本 (C_G)'},
        'param2': {'type': 'net_benefit', 'name': 'PH_BI_CcertI_PO', 'range': np.linspace(-1.0, 3.0, 20), 'label': '政府建立体系下长期投资净收益 (P_H + B_I - C_certI - P_O)', 'vary_component': 'P_H'},
        'title': '政府认证体系成本 vs. 政府建立体系下长期投资净收益'
    },
     {
        'name': 'GovSocialBenefit_vs_GovCost',
        'param1': {'type': 'param', 'name': 'S_G', 'range': np.linspace(0.0, 3.0, 20), 'label': '政府社会效益 (S_G)'},
        'param2': {'type': 'param', 'name': 'C_G', 'range': np.linspace(0.5, 2.5, 20), 'label': '政府认证体系成本 (C_G)'},
        'title': '政府社会效益 vs. 政府认证体系成本'
    },
    {
        'name': 'GovSocialBenefit_vs_EntCertNetBenefit',
        'param1': {'type': 'param', 'name': 'S_G', 'range': np.linspace(0.0, 3.0, 20), 'label': '政府社会效益 (S_G)'},
        'param2': {'type': 'net_benefit', 'name': 'BE_CcertE', 'range': np.linspace(-1.0, 3.0, 20), 'label': '企业认证净收益 (B_E - C_certE)', 'vary_component': 'B_E'},
        'title': '政府社会效益 vs. 企业认证净收益'
    },
    {
        'name': 'GovSocialBenefit_vs_InvLongTermNetBenefit_GovCert',
        'param1': {'type': 'param', 'name': 'S_G', 'range': np.linspace(0.0, 3.0, 20), 'label': '政府社会效益 (S_G)'},
        'param2': {'type': 'net_benefit', 'name': 'PH_BI_CcertI_PO', 'range': np.linspace(-1.0, 3.0, 20), 'label': '政府建立体系下长期投资净收益 (P_H + B_I - C_certI - P_O)', 'vary_component': 'P_H'},
        'title': '政府社会效益 vs. 政府建立体系下长期投资净收益'
    },
    {
        'name': 'EntCertNetBenefit_vs_InvShortTerm',
        'param1': {'type': 'net_benefit', 'name': 'BE_CcertE', 'range': np.linspace(-1.0, 3.0, 20), 'label': '企业认证净收益 (B_E - C_certE)', 'vary_component': 'B_E'},
        'param2': {'type': 'param', 'name': 'P_O', 'range': np.linspace(1.0, 3.0, 20), 'label': '投资机构短期投资收益 (P_O)'},
        'title': '企业认证净收益 vs. 投资机构短期投资收益'
    },
    {
        'name': 'InvLongTermNetBenefit_GovCert_vs_EntStrategic',
        'param1': {'type': 'net_benefit', 'name': 'PH_BI_CcertI_PO', 'range': np.linspace(-1.0, 3.0, 20), 'label': '政府建立体系下长期投资净收益 (P_H + B_I - C_certI - P_O)', 'vary_component': 'P_H'},
        'param2': {'type': 'param', 'name': 'R_O', 'range': np.linspace(0.5, 2.0, 20), 'label': '企业策略性创新收益 (R_O)'},
        'title': '政府建立体系下长期投资净收益 vs. 企业策略性创新收益'
    },
    {
        'name': 'EntCertNetBenefit_vs_InvLongTermNetBenefit_NoGovCert',
        'param1': {'type': 'net_benefit', 'name': 'BE_CcertE', 'range': np.linspace(-1.0, 3.0, 20), 'label': '企业认证净收益 (B_E - C_certE)', 'vary_component': 'B_E'},
        'param2': {'type': 'net_benefit', 'name': 'PL_PO', 'range': np.linspace(-1.0, 2.0, 20), 'label': '政府不建立体系下长期投资净收益 (P_L - P_O)', 'vary_component': 'P_L'},
        'title': '企业认证净收益 vs. 政府不建立体系下长期投资净收益'
    }
]


# 创建二维参数空间图保存目录
param_space_save_dir = os.path.join(BASE_SAVE_DIR, 'important_parameter_space')
os.makedirs(param_space_save_dir, exist_ok=True)

# 固定其他参数 (使用与单因素分析相同的基准参数集)
base_params_2d = get_parameters('sensitivity_base')
# 选择一个初始条件
initial_state_2d = [0.5, 0.5, 0.5]

for config_2d in important_2d_configs:
    config_name = config_2d['name']
    param1_config = config_2d['param1']
    param2_config = config_2d['param2']
    plot_title_2d = config_2d['title'] + f' (初始状态: {initial_state_2d})'

    param1_name = param1_config['name']
    param1_range = param1_config['range']
    param1_label = param1_config['label']
    param1_type = param1_config['type']
    param1_vary_component = param1_config.get('vary_component') # None for type 'param'

    param2_name = param2_config['name']
    param2_range = param2_config['range']
    param2_label = param2_config['label']
    param2_type = param2_config['type']
    param2_vary_component = param2_config.get('vary_component') # None for type 'param'


    print(f"\n--- 对政策组合 '{config_name}' 进行二维敏感性分析 ---")
    print(f"组合: {param1_label} vs. {param2_label}")

    parameter_space_results = []

    # 使用 tqdm 显示进度条
    param_iterator_2d = tqdm(itertools.product(param1_range, param2_range),
                          total=len(param1_range) * len(param2_range),
                          desc=f"Scanning 2D Space: {param1_name} vs {param2_name}")

    for param1_val, param2_val in param_iterator_2d:
        current_params = base_params_2d.copy()

        # Adjust param1
        if param1_type == 'param':
            current_params[param1_name] = param1_val
        elif param1_type == 'net_benefit':
            # 计算当前净收益与目标净收益的差值，并调整指定组成参数
            current_net_benefit = 0
            if param1_name == 'BE_CcertE':
                 current_net_benefit = base_params_2d['B_E'] - base_params_2d['C_certE']
            elif param1_name == 'PH_BI_CcertI_PO':
                 current_net_benefit = base_params_2d['P_H'] + base_params_2d['B_I'] - base_params_2d['C_certI'] - base_params_2d['P_O']
            elif param1_name == 'PL_PO':
                 current_net_benefit = base_params_2d['P_L'] - base_params_2d['P_O']
            # Add other net benefit calculations here if needed
            delta = param1_val - current_net_benefit
            current_params[param1_vary_component] = base_params_2d[param1_vary_component] + delta # Adjust the component from base value

        # Adjust param2
        # IMPORTANT: Recalculate base net benefit for param2 based on potentially adjusted current_params
        # This ensures the two parameters are varied relative to the *same* base state in each grid cell
        # However, for simplicity and standard 2D sensitivity, we usually vary relative to the *original* base_params_2d
        # Let's stick to varying relative to the original base_params_2d for standard interpretation
        if param2_type == 'param':
            current_params[param2_name] = param2_val
        elif param2_type == 'net_benefit':
            current_net_benefit = 0
            if param2_name == 'BE_CcertE':
                 current_net_benefit = base_params_2d['B_E'] - base_params_2d['C_certE']
            elif param2_name == 'PH_BI_CcertI_PO':
                 current_net_benefit = base_params_2d['P_H'] + base_params_2d['B_I'] - base_params_2d['C_certI'] - base_params_2d['P_O']
            elif param2_name == 'PL_PO':
                 current_net_benefit = base_params_2d['P_L'] - base_params_2d['P_O']
            # Add other net benefit calculations here if needed
            delta = param2_val - current_net_benefit
            current_params[param2_vary_component] = base_params_2d[param2_vary_component] + delta # Adjust the component from base value


        sol = solve_replicator_dynamics(initial_state_2d, current_params, t_span)
        final_ess = get_final_state(sol)
        parameter_space_results.append((param1_val, param2_val, final_ess))

    # 绘制并保存 ESS in Parameter Space 图
    save_filename_2d = f'{config_name}_ParameterSpace.pdf'
    save_path_2d = os.path.join(param_space_save_dir, save_filename_2d)

    plot_parameter_space(parameter_space_results, param1_label, param2_label, plot_title_2d, save_path=save_path_2d)






Matplotlib 中文字体设置成功。
绘图前准备代码已完成：模型参数定义、复制动态方程、仿真求解函数、最终状态判断函数、绘图函数（支持保存和只显示出现的ESS）。
接下来我们将进行仿真并绘制结果图。

--- 6. 重要政策工具组合影响的二维参数空间分析 ---

--- 对政策组合 'EntCertNetBenefit_vs_InvLongTermNetBenefit_GovCert' 进行二维敏感性分析 ---
组合: 企业认证净收益 (B_E - C_certE) vs. 政府建立体系下长期投资净收益 (P_H + B_I - C_certI - P_O)


Scanning 2D Space: BE_CcertE vs PH_BI_CcertI_PO: 100%|███████████████████████████| 400/400 [00:02<00:00, 144.77it/s]


图已保存至: simulation_plots\important_parameter_space\EntCertNetBenefit_vs_InvLongTermNetBenefit_GovCert_ParameterSpace.pdf

--- 对政策组合 'EntStrategic_vs_InvShortTerm' 进行二维敏感性分析 ---
组合: 企业策略性创新收益 (R_O) vs. 投资机构短期投资收益 (P_O)


Scanning 2D Space: R_O vs P_O: 100%|█████████████████████████████████████████████| 400/400 [00:02<00:00, 157.34it/s]


图已保存至: simulation_plots\important_parameter_space\EntStrategic_vs_InvShortTerm_ParameterSpace.pdf

--- 对政策组合 'GovCost_vs_EntCertNetBenefit' 进行二维敏感性分析 ---
组合: 政府认证体系成本 (C_G) vs. 企业认证净收益 (B_E - C_certE)


...

图已保存至: simulation_plots\important_parameter_space\EntCertNetBenefit_vs_InvShortTerm_ParameterSpace.pdf

--- 对政策组合 'InvLongTermNetBenefit_GovCert_vs_EntStrategic' 进行二维敏感性分析 ---
组合: 政府建立体系下长期投资净收益 (P_H + B_I - C_certI - P_O) vs. 企业策略性创新收益 (R_O)


Scanning 2D Space: PH_BI_CcertI_PO vs R_O: 100%|█████████████████████████████████| 400/400 [00:02<00:00, 167.47it/s]


图已保存至: simulation_plots\important_parameter_space\InvLongTermNetBenefit_GovCert_vs_EntStrategic_ParameterSpace.pdf

--- 对政策组合 'EntCertNetBenefit_vs_InvLongTermNetBenefit_NoGovCert' 进行二维敏感性分析 ---
组合: 企业认证净收益 (B_E - C_certE) vs. 政府不建立体系下长期投资净收益 (P_L - P_O)


Scanning 2D Space: BE_CcertE vs PL_PO: 100%|█████████████████████████████████████| 400/400 [00:02<00:00, 171.82it/s]


图已保存至: simulation_plots\important_parameter_space\EntCertNetBenefit_vs_InvLongTermNetBenefit_NoGovCert_ParameterSpace.pdf
